# Web Scraping using Python (and Beautiful Soup)

- Web scraping is a term used to describe the use of a program or algorithm to extract and process large amounts of data from the web. 

- Whether you are a data scientist, engineer, or anybody who analyzes large amounts of datasets, the ability to scrape data from the web is a useful skill to have. 

- Let's say you find data from the web, and there is no direct way to download it, web scraping using Python is a skill you can use to extract the data into a useful form that can be imported.

In this session, you will learn about the following:

- Data extraction from the web using Python's Beautiful Soup module
- Data manipulation and cleaning using Python's Pandas library
- Data visualization using Python's Matplotlib library

The dataset used in this tutorial was taken from a 10K race that took place in Hillsboro, OR on June 2017. Specifically, you will analyze the performance of the 10K runners and answer questions such as:

- What was the average finish time for the runners?
- Did the runners' finish times follow a normal distribution?
- Were there any performance differences between males and females of various age groups?

In [10]:
# Importing the necessary modules (pandas, numpy, matplotlib.pyplot, seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [17]:
# To perform web scraping, you should also import the libraries shown below. 
# The urllib.request module is used to open URLs. 
# The Beautiful Soup package is used to extract data from html files. 
# The Beautiful Soup library's name is bs4 which stands for Beautiful Soup, version 4.

from urllib.request import urlopen
from bs4 import BeautifulSoup

In [18]:
# Getting the html of the page is just the first step.
# After importing necessary modules,specify the URL containing the dataset and pass it to urlopen() to get the html of the page.
url = "http://www.hubertiming.com/results/2017GPTR10K"
html = urlopen(url)
html

In [19]:
# Next step is to create a Beautiful Soup object from the html. 
# This is done by passing the html to the BeautifulSoup() function. 
# The Beautiful Soup package is used to parse the html, that is, take the raw html text and break it into Python objects. 
# The second argument 'lxml' is the html parser whose details you do not need to worry about at this point.

soup = BeautifulSoup(html, 'lxml')
type(soup)


bs4.BeautifulSoup

In [20]:
bs4.BeautifulSoup


bs4.BeautifulSoup

In [21]:
# The soup object allows you to extract interesting information about the website you're scraping such as getting the title of the page as shown below.

# Get the title
title = soup.title
print(title)


<title>Race results for the 2017 Intel Great Place to Run \ Urban Clash Games!</title>


In [ ]:
# You can also get the text of the webpage and quickly print it out to check if it is what you expect.

# Print out the text
text = soup.get_text()
print(soup.text)



In [ ]:
"""
You can use the find_all() method of soup to extract useful html tags within a webpage. 
Examples of useful tags include < a > for hyperlinks, < table > for tables, < tr > for table rows, < th > for table headers, and < td > for table cells. 
The code below shows how to extract all the hyperlinks within the webpage.

"""

In [ ]:
soup.find_all('a')

In [ ]:
# As you can see from the output above, html tags sometimes come with attributes such as class, src, etc. 
# These attributes provide additional information about html elements. 
# You can use a for loop and the get('"href") method to extract and print out only hyperlinks.

all_links = soup.find_all("a")
for link in all_links:
    print(link.get("href"))

In [ ]:
# To print out table rows only, pass the 'tr' argument in soup.find_all().

# Print the first 10 rows for sanity check
rows = soup.find_all('tr')
print(rows[:10])

In [ ]:
"""
The goal of this tutorial is to take a table from a webpage and convert it into a dataframe for easier manipulation using Python. 
To get there, you should get all table rows in list form first and then convert that list into a dataframe. 
Below is a for loop that iterates through table rows and prints out the cells of the rows.

"""

In [ ]:
for row in rows:
    row_td = row.find_all('td')
print(row_td)
type(row_td)


The output above shows that each row is printed with html tags embedded in each row. This is not what you want. You can use remove the html tags using Beautiful Soup or regular expressions.

The easiest way to remove html tags is to use Beautiful Soup, and it takes just one line of code to do this. Pass the string of interest into BeautifulSoup() and use the get_text() method to extract the text without html tags.

In [ ]:
str_cells = str(row_td)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()
print(cleantext)


Using regular expressions is highly discouraged since it requires several lines of code and one can easily make mistakes. It requires importing the re (for regular expressions) module. The code below shows how to build a regular expression that finds all the characters inside the < td > html tags and replace them with an empty string for each table row. First, you compile a regular expression by passing a string to match to re.compile(). The dot, star, and question mark (.*?) will match an opening angle bracket followed by anything and followed by a closing angle bracket. It matches text in a non-greedy fashion, that is, it matches the shortest possible string. If you omit the question mark, it will match all the text between the first opening angle bracket and the last closing angle bracket. After compiling a regular expression, you can use the re.sub() method to find all the substrings where the regular expression matches and replace them with an empty string. The full code below generates an empty list, extract text in between html tags for each row, and append it to the assigned list.

In [ ]:
import re

list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
print(clean2)
type(clean2)

The next step is to convert the list into a dataframe and get a quick view of the first 10 rows using Pandas.





In [ ]:
df = pd.DataFrame(list_rows)
df.head(10)


# Data Manipulation and Cleaning


The dataframe is not in the format we want. To clean it up, you should split the "0" column into multiple columns at the comma position. This is accomplished by using the str.split() method.




In [ ]:
df1 = df[0].str.split(',', expand=True)
df1.head(10)


This looks much better, but there is still work to do. The dataframe has unwanted square brackets surrounding each row. You can use the strip() method to remove the opening square bracket on column "0."




In [ ]:
df1[0] = df1[0].str.strip('[')
df1.head(10)


The table is missing table headers. You can use the find_all() method to get the table headers.




In [ ]:
col_labels = soup.find_all('th')


Similar to table rows, you can use Beautiful Soup to extract text in between html tags for table headers.




In [ ]:
all_header = []
col_str = str(col_labels)
cleantext2 = BeautifulSoup(col_str, "lxml").get_text()
all_header.append(cleantext2)
print(all_header)


You can then convert the list of headers into a pandas dataframe.




In [ ]:
df2 = pd.DataFrame(all_header)
df2.head()


Similarly, you can split column "0" into multiple columns at the comma position for all rows.




In [ ]:
df3 = df2[0].str.split(',', expand=True)
df3.head()


The two dataframes can be concatenated into one using the concat() method as illustrated below.




In [ ]:
frames = [df3, df1]

df4 = pd.concat(frames)
df4.head(10)


Below shows how to assign the first row to be the table header.




In [ ]:
df5 = df4.rename(columns=df4.iloc[0])
df5.head()


At this point, the table is almost properly formatted. For analysis, you can start by getting an overview of the data as shown below.




In [ ]:
df5.info()
df5.shape


In [ ]:
df6 = df5.dropna(axis=0, how='any')


The table has 597 rows and 14 columns. You can drop all rows with any missing values.




Also, notice how the table header is replicated as the first row in df5. It can be dropped using the following line of code.




In [ ]:
df7 = df6.drop(df6.index[0])
df7.head()


You can perform more data cleaning by renaming the '[Place' and ' Team]' columns. Python is very picky about space. Make sure you include space after the quotation mark in ' Team]'.




In [ ]:
df7.rename(columns={'[Place': 'Place'},inplace=True)
df7.rename(columns={' Team]': 'Team'},inplace=True)
df7.head()


The final data cleaning step involves removing the closing bracket for cells in the "Team" column.




In [ ]:
df7['Team'] = df7['Team'].str.strip(']')
df7.head()


It took a while to get here, but at this point, the dataframe is in the desired format. Now you can move on to the exciting part and start plotting the data and computing interesting statistics.



# Data Analysis and Visualization


The first question to answer is, what was the average finish time (in minutes) for the runners? You need to convert the column "Chip Time" into just minutes. One way to do this is to convert the column to a list first for manipulation.




In [ ]:
time_list = df7[' Chip Time'].tolist()

# You can use a for loop to convert 'Chip Time' to minutes

time_mins = []
for i in time_list:
    h, m, s = i.split(':')
    math = (int(h) * 3600 + int(m) * 60 + int(s))/60
    time_mins.append(math)
print(time_mins)


The next step is to convert the list back into a dataframe and make a new column ("Runner_mins") for runner chip times expressed in just minutes.




In [ ]:
df7['Runner_mins'] = time_mins
df7.head()


The code below shows how to calculate statistics for numeric columns only in the dataframe.




In [9]:
df7.describe(include=[np.number])


NameError: name 'df7' is not defined

Interestingly, the average chip time for all runners was ~60 mins. The fastest 10K runner finished in 36.35 mins, and the slowest runner finished in 101.30 minutes.

A boxplot is another useful tool to visualize summary statistics (maximum, minimum, medium, first quartile, third quartile, including outliers). Below are data summary statistics for the runners shown in a boxplot. For data visualization, it is convenient to first import parameters from the pylab module that comes with matplotlib and set the same size for all figures to avoid doing it for each figure.

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

In [ ]:
df7.boxplot(column='Runner_mins')
plt.grid(True, axis='y')
plt.ylabel('Chip Time')
plt.xticks([1], ['Runners'])


The second question to answer is: Did the runners' finish times follow a normal distribution?

Below is a distribution plot of runners' chip times plotted using the seaborn library. The distribution looks almost normal.

In [ ]:
x = df7['Runner_mins']
ax = sns.distplot(x, hist=True, kde=True, rug=False, color='m', bins=25, hist_kws={'edgecolor':'black'})
plt.show()


The third question deals with whether there were any performance differences between males and females of various age groups. Below is a distribution plot of chip times for males and females.




In [ ]:
f_fuko = df7.loc[df7[' Gender']==' F']['Runner_mins']
m_fuko = df7.loc[df7[' Gender']==' M']['Runner_mins']
sns.distplot(f_fuko, hist=True, kde=True, rug=False, hist_kws={'edgecolor':'black'}, label='Female')
sns.distplot(m_fuko, hist=False, kde=True, rug=False, hist_kws={'edgecolor':'black'}, label='Male')
plt.legend()

The distribution indicates that females were slower than males on average. You can use the groupby() method to compute summary statistics for males and females separately as shown below.




In [ ]:
g_stats = df7.groupby(" Gender", as_index=True).describe()
print(g_stats)


The average chip time for all females and males was ~66 mins and ~58 mins, respectively. Below is a side-by-side boxplot comparison of male and female finish times.




In [ ]:
df7.boxplot(column='Runner_mins', by=' Gender')
plt.ylabel('Chip Time')
plt.suptitle("")


In this tutorial, you performed web scraping using Python. You used the Beautiful Soup library to parse html data and convert it into a form that can be used for analysis. You performed cleaning of the data in Python and created useful plots (box plots, bar plots, and distribution plots) to reveal interesting trends using Python's matplotlib and seaborn libraries. After this tutorial, you should be able to use Python to easily scrape data from the web, apply cleaning techniques and extract useful insights from the data.

